In [1]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import os
import pandas as pd
from multiprocessing import Pool
import json

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [2]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_7s2ZAfrdbfGPDDQtPucaicPGsP7o



### Create list of alpha expressions
#### Step 1. Download datsets

In [ ]:
# 保存和加载字典的函数
def save_progress(progress, filename='progress.json'):
    with open(filename, 'w') as f:
        json.dump(progress, f)

def load_progress(filename='progress.json'):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return {}

In [ ]:
datasets_df = hf.get_datasets(s) # by default we load all datasets USA TOP3000 delay 1
# datasets_df.tail() # DataFrame.head() shows first 5 rows of the dataframe 
# print(len(datasets_df))

# 筛选潜在数据集
potential_datasets_df = datasets_df[
    (datasets_df["delay"] == 1) &
    (datasets_df["coverage"] > 0.8) & (datasets_df["coverage"] <= 1) &
    (datasets_df["fieldCount"] > 0) & (datasets_df["fieldCount"] < 3000) &
    (datasets_df["region"] == 'GLB') &
    (datasets_df["universe"] == 'TOP3000')
    # (datasets_df["userCount"] > 0) & (datasets_df["userCount"] < 100) &
    # (datasets_df["valueScore"] > 1) & (datasets_df["valueScore"] < 10)
]

print(len(potential_datasets_df))

104


#### Step 2. 对每个潜在数据集挖掘因子

In [ ]:
def simulate_factors(dataset_id, progress):
    try:
        # 检查是否已经有模拟结果
        result_file = f'results_{dataset_id}.csv'
        if os.path.exists(result_file):
            print(f'Results for dataset {dataset_id} already exist. Skipping...')
            progress[dataset_id] = 'completed'
            save_progress(progress)
            return
        
        # 下载数据集字段
        datafields_df = hf.get_datafields(s, dataset_id=dataset_id)
        
        # 生成因子表达式
        expression_list = [f'ts_skewness(vec_avg({x}),120)' for x in datafields_df.id.values.tolist()]
        
        # 生成alpha列表
        alpha_list = [ace.generate_alpha(x, region= "USA", universe = "TOP3000",) for x in expression_list]
        
        # 模拟alpha列表
        result = ace.simulate_alpha_list_multi(s, alpha_list)
        
        # 美化结果
        result_st1 = hf.prettify_result(result, clickable_alpha_id=False)
        
        # 保存结果
        result_st1.to_csv(result_file)
        print(f'Results for dataset {dataset_id} processed successfully.')
        
        # 更新进度
        progress[dataset_id] = 'completed'
        save_progress(progress)
        
    except Exception as e:
        print(f'Error processing dataset {dataset_id}: {e}')
        # 更新进度为错误
        progress[dataset_id] = 'error'
        save_progress(progress)


In [ ]:
print(len(potential_datasets_df.id.values))

104


In [ ]:
# 加载进度字典
progress = load_progress()

# 过滤掉已经处理过的数据集
dataset_ids = [dataset_id for dataset_id in potential_datasets_df.id.values.tolist() if dataset_id not in progress]
batch_size = 10  # 每批处理的数据集数量
batches = [dataset_ids[i:i + batch_size] for i in range(0, len(dataset_ids), batch_size)]

# 逐批处理数据集
def run_simulation():
    for batch in batches:
        for dataset_id in batch:
            simulate_factors(dataset_id, progress=progress)

# 运行模拟
run_simulation()
 

  0%|          | 2/846 [00:29<2:51:10, 12.17s/it]

An error occurred
An error occurred


  0%|          | 3/846 [00:30<1:36:03,  6.84s/it]

An error occurred


  1%|          | 5/846 [00:53<2:00:36,  8.60s/it]

...

 14%|█▍        | 54/385 [07:14<45:21,  8.22s/it]

An error occurred


 14%|█▍        | 55/385 [07:19<40:16,  7.32s/it]

An error occurred
Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_JryQQUcztYiWrgASMUP9KkbDfCRD



: 

#### Step 3. 合并和分析结果

In [ ]:
import glob

# 读取所有结果文件
all_results = pd.concat([pd.read_csv(f) for f in glob.glob('results_*.csv')])

# 进一步分析合并后的结果
# 例如筛选出表现最好的因子
top_factors = all_results.sort_values(by='sharpe', ascending=False)
top_factors


,Unnamed: 0,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,...,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_competition,matches_themes
0,0,408142,20000000,107,0,0.5000,2.5509,0.0254,0.010204,11.23,...,"ts_skewness(vec_avg(nws17_d1_event_text),120)",FAIL,PASS,FAIL,PASS,PASS,FAIL,PASS,WARNING,WARNING
0,0,420984,20000000,0,1,0.1667,0.8771,0.0219,0.010525,9.82,...,ts_skewness(vec_avg(fnd14_td_nve_cns_tda_rpr_t...,FAIL,PASS,NaN,PASS,PASS,ERROR,PASS,WARNING,WARNING
1,1,20672148,20000000,11,12,0.1197,0.5933,0.2776,0.009917,3.42,...,"ts_skewness(vec_avg(fnd14_bus_state),120)",FAIL,PASS,NaN,PASS,FAIL,PASS,PASS,WARNING,WARNING
2,2,20500490,20000000,11,11,0.1192,0.5884,0.2776,0.009872,3.38,...,"ts_skewness(vec_avg(fnd14_loc_state),120)",FAIL,PASS,NaN,PASS,FAIL,PASS,PASS,WARNING,WARNING
1,1,2482578,20000000,711,628,0.0839,0.0678,0.0486,0.001616,1.15,...,"ts_skewness(vec_avg(pv64_cins_dif_rcode),120)",FAIL,PASS,NaN,PASS,FAIL,PASS,PASS,WARNING,WARNING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,179,-14967246,20000000,33,35,0.1300,-0.4223,1.7037,-0.006499,-2.07,...,ts_skewness(vec_avg(fnd14_last_reported_audito...,FAIL,PASS,NaN,FAIL,FAIL,PASS,PASS,WARNING,WARNING
174,174,-9625188,20000000,31,32,0.1871,-0.2716,1.0206,-0.002904,-1.40,...,ts_skewness(vec_avg(fnd14_last_known_auditor_f...,FAIL,PASS,NaN,FAIL,FAIL,FAIL,PASS,WARNING,WARNING
180,180,-15467294,20000000,33,34,0.1299,-0.4364,1.7812,-0.006720,-2.18,...,ts_skewness(vec_avg(fnd14_last_known_auditor_n...,FAIL,PASS,NaN,FAIL,FAIL,PASS,PASS,WARNING,WARNING
177,177,-9036344,20000000,848,760,0.1406,-0.2558,1.1000,-0.003639,-1.67,...,ts_skewness(vec_avg(fnd14_latestfy_filing_type...,FAIL,PASS,NaN,FAIL,FAIL,FAIL,PASS,WARNING,WARNING


### Change the expression - what would you do to improve alpha's results?
<br>Use your own logic to improve the alphas.</br>
<br>Here we will apply **group_rank** to overcome poor weight distribution.</br>

In [ ]:
prospect_alphas = top_factors.loc[lambda x: x.fitness>0.2]['expression'].values
prospect_alphas
new_expression_list = ['group_rank(' + a + ',sector)' for a in prospect_alphas]
new_expression_list

['group_rank(ts_skewness(vec_avg(nws17_d1_event_text),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_td_nve_cns_tda_rpr_tsl),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_bus_state),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_loc_state),120),sector)',
 'group_rank(ts_skewness(vec_avg(pv64_cins_dif_rcode),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_bus_state_name),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_loc_state_name),120),sector)',
 'group_rank(ts_skewness(vec_avg(pv64_out_stal_fund_ira_invest),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_bus_county_name),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_mail_city),120),sector)',
 'group_rank(ts_skewness(vec_avg(pv64_px_stal_px_ask),120),sector)',
 'group_rank(ts_skewness(vec_avg(mws50_ghc_lna),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_latest_tax_fees_sum),120),sector)',
 'group_rank(ts_skewness(vec_avg(fnd14_mail_state),120),sector)',
 'group_rank(ts_skewness(v

In [ ]:
#generating new simulation data for new expressions

new_alpha_list = [ace.generate_alpha(x, region= "USA", universe = "TOP3000",) for x in new_expression_list]

new_result = ace.simulate_alpha_list_multi(s, new_alpha_list)

  5%|▍         | 2/41 [01:34<30:34, 47.03s/it]


KeyboardInterrupt: 

In [ ]:
result_st2 = hf.prettify_result(new_result, clickable_alpha_id=False)
result_st2

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,...,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_competition,matches_themes
0,408140,20000000,107,0,0.5000,2.5509,0.0254,0.010204,11.23,4.97,...,group_rank(ts_skewness(vec_avg(nws17_d1_event_...,FAIL,PASS,FAIL,PASS,PASS,FAIL,PASS,WARNING,WARNING
1,420984,20000000,0,1,0.1667,0.8771,0.0219,0.010525,9.82,4.28,...,group_rank(ts_skewness(vec_avg(fnd14_td_nve_cn...,FAIL,PASS,FAIL,PASS,PASS,ERROR,PASS,WARNING,WARNING
2,8829271,20000000,31,31,0.1211,0.2520,0.1800,0.004160,1.86,1.31,...,group_rank(ts_skewness(vec_avg(fnd14_mail_city...,FAIL,PASS,FAIL,PASS,FAIL,FAIL,PASS,WARNING,WARNING
3,10271130,20000000,16,15,0.1189,0.2935,0.2545,0.004938,1.81,1.18,...,group_rank(ts_skewness(vec_avg(fnd14_bus_count...,FAIL,PASS,FAIL,PASS,FAIL,FAIL,PASS,WARNING,WARNING
4,8562851,20000000,26,26,0.1191,0.2444,0.1651,0.004102,1.72,1.23,...,group_rank(ts_skewness(vec_avg(fnd14_bus_city)...,FAIL,PASS,FAIL,PASS,FAIL,FAIL,PASS,WARNING,WARNING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,-1131232,20000000,1127,1121,0.0633,-0.0113,0.1456,-0.000358,-0.13,-0.42,...,"group_rank(ts_skewness(vec_avg(nws17_d1_bee),1...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING,WARNING
119,-1576360,20000000,1131,1123,0.0550,-0.0186,0.3112,-0.000678,-0.17,-0.45,...,group_rank(ts_skewness(vec_avg(pv64_dif_min_su...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING,WARNING
120,-774695,20000000,957,939,0.0961,-0.0219,0.2095,-0.000455,-0.17,-0.40,...,group_rank(ts_skewness(vec_avg(fnd14_latestfy_...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING,WARNING
121,-1607923,20000000,407,493,0.0639,-0.0186,0.1757,-0.000582,-0.22,-0.58,...,group_rank(ts_skewness(vec_avg(pv64_out_stal_f...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING,WARNING


In [ ]:
## We will join before and after dataframes by field

top_factors['field'] = top_factors['expression'].apply(lambda st: st[st.find("(vec_avg(")+1:st.find(",")]).copy()
result_st2['field'] = result_st2['expression'].apply(lambda st: st[st.find("(vec_avg(")+1:st.find(",")]).copy()

#merging results before and after improvement

compare_results = pd.merge(top_factors, result_st2, on='field', suffixes=('_before', '_after'))

#selecting column list returned by merging the two alpha results

col_list = ['fitness_before', 'fitness_after','sharpe_before', 'sharpe_after', 'alpha_id_before', 'alpha_id_after',
                 'expression_before', 'expression_after']

compare_results[col_list]\
    .style.format({'alpha_id_before': hf.make_clickable_alpha_id, 'alpha_id_after': hf.make_clickable_alpha_id})

,fitness_before,fitness_after,sharpe_before,sharpe_after,alpha_id_before,alpha_id_after,expression_before,expression_after
0,11.230000,11.230000,4.970000,4.970000,g1YNX0l,lVdV9RO,"ts_skewness(vec_avg(nws17_d1_event_text),120)","group_rank(ts_skewness(vec_avg(nws17_d1_event_text),120),sector)"
1,9.820000,9.820000,4.280000,4.280000,8zmWRYX,27M7WnZ,"ts_skewness(vec_avg(fnd14_td_nve_cns_tda_rpr_tsl),120)","group_rank(ts_skewness(vec_avg(fnd14_td_nve_cns_tda_rpr_tsl),120),sector)"
2,3.420000,1.250000,1.570000,0.870000,wR87RY5,qA8A3mP,"ts_skewness(vec_avg(fnd14_bus_state),120)","group_rank(ts_skewness(vec_avg(fnd14_bus_state),120),sector)"
3,3.380000,1.270000,1.560000,0.880000,dx2JOPj,Ew1wLY0,"ts_skewness(vec_avg(fnd14_loc_state),120)","group_rank(ts_skewness(vec_avg(fnd14_loc_state),120),sector)"
4,1.150000,0.870000,1.560000,1.340000,OpOjZPb,g16VgRQ,"ts_skewness(vec_avg(pv64_cins_dif_rcode),120)","group_rank(ts_skewness(vec_avg(pv64_cins_dif_rcode),120),sector)"
5,3.360000,1.170000,1.550000,0.830000,bLY0LxM,17e7P1J,"ts_skewness(vec_avg(fnd14_bus_state_name),120)","group_rank(ts_skewness(vec_avg(fnd14_bus_state_name),120),sector)"
6,3.350000,1.180000,1.550000,0.840000,ro1a5w1,eOmOvgN,"ts_skewness(vec_avg(fnd14_loc_state_name),120)","group_rank(ts_skewness(vec_avg(fnd14_loc_state_name),120),sector)"
7,1.840000,1.510000,1.310000,1.140000,5wxd0r5,9wYOdNx,"ts_skewness(vec_avg(pv64_out_stal_fund_ira_invest),120)","group_rank(ts_skewness(vec_avg(pv64_out_stal_fund_ira_invest),120),sector)"
8,1.560000,1.810000,1.110000,1.180000,oK1jK6b,eOmOvLz,"ts_skewness(vec_avg(fnd14_bus_county_name),120)","group_rank(ts_skewness(vec_avg(fnd14_bus_county_name),120),sector)"
9,1.160000,1.860000,1.050000,1.310000,qAK20bO,wRqRPL2,"ts_skewness(vec_avg(fnd14_mail_city),120)","group_rank(ts_skewness(vec_avg(fnd14_mail_city),120),sector)"


### Check merged alpha performance

run the following code to do a before and after comparison of your merged pool post alpha submission

In [ ]:
performance_comparison = ace.performance_comparison(s, result_st2['alpha_id'][0])

#### Step 4. 提交和监控

In [ ]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(new_result)
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

for alpha_id in passed_alphas:
    submit_result = submit_alpha(s, alpha_id)
    print(f'Alpha {alpha_id} submitted: {submit_result}')


Failed alphas:['Ew1wLY0' 'g16VgRQ' '17e7P1J' 'lVdV9RO' '27M7WnZ' 'qA8A3mP' 'Yp6pJ9J'
 'roYoVno' 'qA8A3p1' 'eOmOvgN' '9wYOdNx' 'eOmOvLz' 'wRqRPL2' 'Zp6vzbY'
 'qA8A3g2' 'OpOpg0v' 'WpMp0vG' '17e7Pr6' 'ad6dGr9' 'dxpxv5X' 'QaMpzRK'
 'Kb3bZX8' 'Yp6pY2v' 'LpWYrea' 'XpGAenm' 'bLXLAor' 'RpMpPda' '5wxderN'
 '37w7Jze' 'Gwq52J5' 'm8pWQ51' 'WpMpJlj' 'k3d39ak' 'k3d3Oz8' 'LpWY2Wv'
 '9wYO2vq' 'JpqJ2oA' 'z9Z9LKO' 'qA8AV7A' 'xxqxEOb' 'XpGAOel' 'wRqRvql'
 '8zbz26z' '17e72mm' 'eOmOW26' 'Gwq52k0' 'EwZPjG1' '7wWRRk2' 'LpWp2V9'
 'bLX11o6' 'Aw988gE' 'Np0p2Ke' 'VpVbbX8' 'bLXLKxK' 'xxqxEpw' 'Aw9w5dw'
 'roYonMJ' 'oK7KkWJ' 'LpWp2Z2' 'Zp6p1mY' '17eAAAK' '5wxw2Gk' '37wvvmZ'
 'qA8ApZP' 'xxqxoab' 'WpMpx2P' 'Kb3bp0N' 'VpVbQEV' 'Kb3MA6j' 'p6me2qo'
 'ad690rO' 'LpWYvPM' 'Zp6v83j' 'PpNmRa7' 'XpGA3Qz' 'm8pWGj2' 'LpWYv7n'
 'dxprMjK' 'z9ZnxrG' 'vL8XEOb' 'OpOXLA1' 'eOmZY1M' 'XpGA3Vm' 'GwP52XJ'
 'qA8JR1Z' 'dxprGXE' '17eAM9z' 'xxqpZjp' 'roYMx51' 'Kb3M1Oj' 'ad695k6'
 'dxprN5g' 'OpOXl0b' 'OpOXlJJ' '8zbxMvX' '8zmx0AV' 'Yp63jKv' 'p